In [14]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [3]:
# Basics
import tensorflow as tf
import pandas as pd
import xarray as xr

# Helpful
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import *  # DRN implementation
from src.models.DRN.DRN_load_score import *  # Load DRN_scores
from src.models.DRN.DRN_load_model import *  # Load Models
from src.models.DRN.DRN_train import *
from src.models.EMOS_local.EMOS_local_load_score import *  # Load EMOS_local_scores
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models#
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores

<IPython.core.display.Javascript object>

In [12]:
# Load all training data and validation data of each variable
train_var_denormed, val_var_denormed = ldpd.load_data_all_train_val_proc_denorm()
# Load all test data of each variable:
test_var_denormed = ldpd.load_data_all_test_proc_denorm()

all_var_denormed = [train_var_denormed, val_var_denormed, test_var_denormed]

<IPython.core.display.Javascript object>

In [ ]:
# Split all 3 datasets into X and y for each lead_time
dat_X_train_lead_all_denorm, dat_y_train_lead_all_denorm = split_var_lead(
    train_var_denormed
)
dat_X_val_lead_all_denorm, dat_y_val_lead_all_denorm = split_var_lead(val_var_denormed)

dat_X_test_lead_all_denorm, dat_y_test_lead_all_denorm = split_var_lead(
    test_var_denormed
)

### 0. Set up

In [63]:
var_names = ["u10", "v10", "t2m", "t850", "z500", "ws10"]

<IPython.core.display.Javascript object>

### 1. Mean_std_min_max

In [53]:
def calc_mean_std_min_max(ds):
    mean = ds.mean(dim=["forecast_date", "lat", "lon"]).values
    std = ds.std(dim=["forecast_date", "lat", "lon"]).values
    min_val = ds.min(dim=["forecast_date", "lat", "lon"]).values
    max_val = ds.max(dim=["forecast_date", "lat", "lon"]).values

    print(f"Mean:{mean}")
    print(f"Std Dev:{std}")
    print(f"Min:{min_val}")
    print(f"Max:{max_val}")

    q25 = ds.quantile(0.25, dim=["forecast_date", "lat", "lon"]).values
    q50 = ds.quantile(0.5, dim=["forecast_date", "lat", "lon"]).values
    q75 = ds.quantile(0.75, dim=["forecast_date", "lat", "lon"]).values

    print(f"25th Percentile:{q25}")
    print(f"Median:{q50}")
    print(f"75th Percentile:{q75} \n")

<IPython.core.display.Javascript object>

#### 1.1 y-values

In [54]:
print("Training set:")
calc_mean_std_min_max(dat_y_train_lead_all_denorm[5][0])

print("Validation set:")
calc_mean_std_min_max(dat_y_val_lead_all_denorm[5][0])

print("Testing set")
calc_mean_std_min_max(dat_y_test_lead_all_denorm[5][0])

Training set:
Mean:5.893629550933838
Std Dev:4.109990119934082
Min:0.00022332968364935368
Max:32.72968292236328
25th Percentile:2.537087619304657
Median:4.962679147720337
75th Percentile:8.470197677612305 

Validation set:
Mean:5.687119007110596
Std Dev:3.918269157409668
Min:0.0007609213935211301
Max:28.609642028808594
25th Percentile:2.5006598830223083
Median:4.792049884796143
75th Percentile:8.162525653839111 

Testing set
Mean:5.656344413757324
Std Dev:3.9310362339019775
Min:0.002409601118415594
Max:31.182769775390625
25th Percentile:2.4133355617523193
Median:4.777425527572632
75th Percentile:8.165741205215454 



<IPython.core.display.Javascript object>

#### 1.2 X_ws10-mean_values

In [55]:
print("Training set:")
calc_mean_std_min_max(dat_X_train_lead_all_denorm[5][0].isel(mean_std=0))

print("Validation set:")
calc_mean_std_min_max(dat_X_val_lead_all_denorm[5][0].isel(mean_std=0))

print("Testing set")
calc_mean_std_min_max(dat_X_test_lead_all_denorm[5][0].isel(mean_std=0))

Training set:
Mean:5.5992207527160645
Std Dev:3.7720556259155273
Min:0.34068697690963745
Max:25.55792808532715
25th Percentile:2.6262742280960083
Median:4.658308267593384
75th Percentile:7.850168585777283 

Validation set:
Mean:5.359326362609863
Std Dev:3.5506463050842285
Min:0.3456885814666748
Max:26.658920288085938
25th Percentile:2.5649388432502747
Median:4.477293968200684
75th Percentile:7.491690993309021 

Testing set
Mean:5.512833118438721
Std Dev:3.72432804107666
Min:0.3354964852333069
Max:27.2584228515625
25th Percentile:2.558821380138397
Median:4.5638344287872314
75th Percentile:7.783245205879211 



<IPython.core.display.Javascript object>

#### 1.3 X_ws10-std_values

In [58]:
print("Training set:")
calc_mean_std_min_max(dat_X_train_lead_all_denorm[5][0].isel(mean_std=1))

print("Validation set:")
calc_mean_std_min_max(dat_X_val_lead_all_denorm[5][0].isel(mean_std=1))

print("Testing set")
calc_mean_std_min_max(dat_X_test_lead_all_denorm[5][0].isel(mean_std=1))

Training set:
Mean:1.093207597732544
Std Dev:0.40825989842414856
Min:0.1504003405570984
Max:4.9724907875061035
25th Percentile:0.7635576277971268
Median:1.0904349088668823
75th Percentile:1.3831287622451782 

Validation set:
Mean:1.095662236213684
Std Dev:0.4061141312122345
Min:0.1685582995414734
Max:5.79463005065918
25th Percentile:0.7629648298025131
Median:1.102105736732483
75th Percentile:1.3864889442920685 

Testing set
Mean:1.0945749282836914
Std Dev:0.40778258442878723
Min:0.16639988124370575
Max:5.280528545379639
25th Percentile:0.762464627623558
Median:1.0955091714859009
75th Percentile:1.3860719203948975 



<IPython.core.display.Javascript object>

#### 1.4 All variables X_mean

In [66]:
for var in range(6):
    print(f"Training set {var_names[var]}:")
    calc_mean_std_min_max(dat_X_train_lead_all_denorm[var][0].isel(mean_std=0))

    print(f"Validation set {var_names[var]}:")
    calc_mean_std_min_max(dat_X_val_lead_all_denorm[var][0].isel(mean_std=0))

    print(f"Testing set {var_names[var]}:")
    calc_mean_std_min_max(dat_X_test_lead_all_denorm[var][0].isel(mean_std=0))

Training set u10:
Mean:0.9774109125137329
Std Dev:4.602171421051025
Min:-21.879791259765625
Max:24.110332489013672
25th Percentile:-1.5342862904071808
Median:0.4389640539884567
75th Percentile:3.264902949333191 

Validation set u10:
Mean:0.6941683292388916
Std Dev:4.327922821044922
Min:-21.4503173828125
Max:24.962617874145508
25th Percentile:-1.580822467803955
Median:0.31287313997745514
75th Percentile:2.822409689426422 

Testing set u10:
Mean:0.7709811329841614
Std Dev:4.5834550857543945
Min:-21.398300170898438
Max:22.742271423339844
25th Percentile:-1.7096210718154907
Median:0.29015499353408813
75th Percentile:2.9986899495124817 

Training set v10:
Mean:0.09877749532461166
Std Dev:4.603442668914795
Min:-22.768943786621094
Max:24.94713020324707
25th Percentile:-2.269433617591858
Median:-0.010042786598205566
75th Percentile:2.3366504907608032 

Validation set v10:
Mean:-0.19723820686340332
Std Dev:4.453178882598877
Min:-22.767797470092773
Max:20.35834503173828
25th Percentile:-2.524662

<IPython.core.display.Javascript object>

#### 1.5 All variablex X_std

In [67]:
for var in range(6):
    print(f"Training set {var_names[var]}:")
    calc_mean_std_min_max(dat_X_train_lead_all_denorm[var][0].isel(mean_std=1))

    print(f"Validation set {var_names[var]}:")
    calc_mean_std_min_max(dat_X_val_lead_all_denorm[var][0].isel(mean_std=1))

    print(f"Testing set {var_names[var]}:")
    calc_mean_std_min_max(dat_X_test_lead_all_denorm[var][0].isel(mean_std=1))

Training set u10:
Mean:1.235317349433899
Std Dev:0.493835985660553
Min:0.1993524432182312
Max:6.521861553192139
25th Percentile:0.8385732769966125
Median:1.2087727189064026
75th Percentile:1.5659054517745972 

Validation set u10:
Mean:1.2378140687942505
Std Dev:0.49094656109809875
Min:0.20069219172000885
Max:5.8617048263549805
25th Percentile:0.8403500467538834
Median:1.2203502655029297
75th Percentile:1.5679243803024292 

Testing set u10:
Mean:1.2375435829162598
Std Dev:0.49333062767982483
Min:0.2005666196346283
Max:6.407730579376221
25th Percentile:0.8387020230293274
Median:1.21567702293396
75th Percentile:1.570154845714569 

Training set v10:
Mean:1.2506605386734009
Std Dev:0.5202142000198364
Min:0.18684841692447662
Max:7.064139366149902
25th Percentile:0.8187906742095947
Median:1.2275455594062805
75th Percentile:1.6073204576969147 

Validation set v10:
Mean:1.2547359466552734
Std Dev:0.5146058201789856
Min:0.2028014063835144
Max:6.44441556930542
25th Percentile:0.8238360732793808
M

<IPython.core.display.Javascript object>

#### 1.6 All variables y

In [68]:
for var in range(6):
    print(f"Training set {var_names[var]}:")
    calc_mean_std_min_max(dat_y_train_lead_all_denorm[var][0])

    print(f"Validation set {var_names[var]}:")
    calc_mean_std_min_max(dat_y_val_lead_all_denorm[var][0])

    print(f"Testing set {var_names[var]}:")
    calc_mean_std_min_max(dat_y_test_lead_all_denorm[var][0])

Training set u10:
Mean:1.1076210737228394
Std Dev:4.896441459655762
Min:-22.825820922851562
Max:25.951784133911133
25th Percentile:-1.6917996406555176
Median:0.6262749433517456
75th Percentile:3.7270898818969727 

Validation set u10:
Mean:0.8186009526252747
Std Dev:4.630218029022217
Min:-21.554235458374023
Max:28.554580688476562
25th Percentile:-1.7322227954864502
Median:0.48217204213142395
75th Percentile:3.2664806842803955 

Testing set u10:
Mean:0.916734516620636
Std Dev:4.882928848266602
Min:-21.924604415893555
Max:26.758827209472656
25th Percentile:-1.8506474494934082
Median:0.46955811977386475
75th Percentile:3.48166823387146 

Training set v10:
Mean:0.005043647717684507
Std Dev:4.860601902008057
Min:-25.78934669494629
Max:26.914140701293945
25th Percentile:-2.6238551139831543
Median:-0.046562761068344116
75th Percentile:2.508944511413574 

Validation set v10:
Mean:-0.30554401874542236
Std Dev:4.714786052703857
Min:-21.748638153076172
Max:22.50701904296875
25th Percentile:-2.8954

<IPython.core.display.Javascript object>

### 1. Results:
Seem very similar through all 3 sets